In [1]:
import beneath
import pandas as pd
import plotly.express as px
from datetime import datetime

In [2]:
BLACKLIST = {
    "FOMO",
    "DD",
    "EOD",
    "TA",
    "PT",
    "RSI",
    "HUGE",
    "ATH",
    "USA",
    "AI",
    "IMO",
    "AM",
    "UK",
    "BIG",
    "SO",
    "OR",
    "FOR",
    "ALL",
    "IT",
    "BE",
    "ARE",
    "NOW",
    "ON",
    "ME",
    "CAN",
    "VERY",
    "SI",
    "TV",
    "BY",
    "NEW",
    "OUT",
    "LOVE",
    "GO",
    "PM",
    "NEXT",
    "ANY",
    "ET",
    "HAS",
    "ONE",
    "PLAY",
    "LOW",
    "III",
    "CASH",
    "RNG",
    "GOOD",
    "REAL",
    "SEE",
    "RE"
}

# Overview

In [9]:
mentions_weekly = await beneath.query_warehouse("""
    SELECT symbol, timestamp_trunc(timestamp, week) as week, count(*) as num_mentions
    FROM `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions`
    GROUP BY symbol, week
    ORDER BY week, num_mentions desc
""")

In [10]:
top_symbols = mentions_weekly.loc[mentions_weekly['num_mentions'] > 100]['symbol'].unique()
top_mentions_weekly = mentions_weekly.loc[mentions_weekly['symbol'].isin(top_symbols)]
fig = px.line(top_mentions_weekly, x="week", y="num_mentions", line_group="symbol", color="symbol", title="Weekly mentions of most popular companies")
fig.show()

# June report

TODO: refactor queries to batch all feature engineering for a given entity (stocks, authors, etc.) in single queries

## Posts

In [ ]:
posts = await beneath.query_warehouse("""
SELECT p.created_on, p.title, p.text, p.author, p.flair, p.permalink, count(c.id) as num_comments
FROM `examples/reddit/r-wallstreetbets-comments` c
JOIN `examples/reddit/r-wallstreetbets-posts` p ON c.post_id = p.id
WHERE timestamp_trunc(p.created_on, month) = timestamp("2021-06-01")
GROUP BY p.created_on, p.title, p.text, p.author, p.flair, p.permalink
""")

Post with most comments

In [379]:
posts.sort_values('num_comments', ascending=False).head(3)

# get the full values:
# posts.sort_values('num_comments', ascending=False)['title'][0]
# posts.sort_values('num_comments', ascending=False)['title'][1]

,created_on,title,text,author,flair,permalink,num_comments,@meta.timestamp
50,2021-06-11 20:00:17+00:00,Weekend Discussion Thread for the Weekend of J...,Your weekend discussion thread. Please keep th...,OPINION_IS_UNPOPULAR,Weekend Discussion,/r/wallstreetbets/comments/nxosm3/weekend_disc...,63166,2021-06-30 09:02:08.950000+00:00
83,2021-06-03 10:00:13+00:00,"Daily Discussion Thread for June 03, 2021",Your daily trading discussion thread. Please k...,OPINION_IS_UNPOPULAR,Daily Discussion,/r/wallstreetbets/comments/nr9r9t/daily_discus...,56242,2021-06-30 09:02:08.950000+00:00
79,2021-06-02 10:00:15+00:00,"Daily Discussion Thread for June 02, 2021",Your daily trading discussion thread. Please k...,OPINION_IS_UNPOPULAR,Daily Discussion,/r/wallstreetbets/comments/nqi9f6/daily_discus...,40962,2021-06-30 09:02:08.950000+00:00


Post with most words

In [ ]:
longest_posts = await beneath.query_warehouse("""
SELECT created_on, id, author, title, text, permalink, flair, length(text) as post_length
FROM `examples/reddit/r-wallstreetbets-posts`
WHERE timestamp_trunc(created_on, month) = timestamp("2021-06-01")
ORDER BY post_length desc
LIMIT 50
""")

Post with the most avg. words per participant

In [381]:
good_discussion_posts = await beneath.query_warehouse("""
SELECT p.created_on, p.title, p.text, p.author, p.flair, p.permalink, 
    count(c.id) as num_comments, sum(length(c.text)) as length_comments, sum(length(c.text))/count(c.id) as avg_length_per_comment
FROM `examples/reddit/r-wallstreetbets-comments` c
JOIN `examples/reddit/r-wallstreetbets-posts` p ON c.post_id = p.id
WHERE timestamp_trunc(p.created_on, month) = timestamp("2021-06-01")
GROUP BY p.created_on, p.title, p.text, p.author, p.flair, p.permalink
ORDER BY avg_length_per_comment desc
""")

<ipython-input-381-71e7c1b5b3ca>:1: RuntimeWarning:

coroutine 'query_warehouse' was never awaited



In [385]:
good_discussion_posts[good_discussion_posts['num_comments'] >= 1].tail()

,created_on,title,text,author,flair,permalink,num_comments,length_comments,avg_length_per_comment,@meta.timestamp
30446,2021-06-28 23:30:14+00:00,SP500 Winners and Losers | 6/28/2021,**Winners** \nWinner of the day by sector | *...,Engineer_Economist,DD,/r/wallstreetbets/comments/o9w47u/sp500_winner...,1,1788,1788.0,2021-06-30 11:27:18.190000+00:00
30447,2021-06-25 04:36:07+00:00,WeedMaps ($MAPS) & Springbig —> Cannabis Loyal...,**Springbig**\n\nSpringbig is an advertising n...,seebz69,DD,/r/wallstreetbets/comments/o7ghrx/weedmaps_map...,2,4003,2001.5,2021-06-30 11:27:18.190000+00:00
30448,2021-06-18 12:14:19+00:00,$SRNE- boom list!,Sorrento Therapeutics Boom List (re-posted) 1....,Beautiful_Law8845,Discussion,/r/wallstreetbets/comments/o2npej/srne_boom_list/,2,4274,2137.0,2021-06-30 11:27:18.190000+00:00
30449,2021-06-09 15:30:41+00:00,"BIONANO GENOMICS INC, SQUEEZE THEM BALLLSSS","Read the comments down below \n\n\n""Credits t...",jayz90210,DD,/r/wallstreetbets/comments/nvyn09/bionano_geno...,2,4708,2354.0,2021-06-30 11:27:18.190000+00:00
30450,2021-06-19 18:31:27+00:00,EDIT - Editas Theraputics is worth a look,EDIT has partnerships to cure Parkinson’s with...,Diwali0,DD,/r/wallstreetbets/comments/o3mnmx/edit_editas_...,4,13422,3355.5,2021-06-30 11:27:18.190000+00:00


In [ ]:
# - Post with most :gem::raised_hands: count (incl. in comments)
# - Post with most :rocket: count (incl. in comments)

## Authors

Author that spurred the most discussion

In [365]:
posts.groupby('author')['num_comments'].sum().reset_index().sort_values('num_comments', ascending=False).head(10)

,author,num_comments
8204,OPINION_IS_UNPOPULAR,1015336
12529,VisualMod,110549
17671,pittluke,8250
19664,yolocallking,4850
13041,Your_Boy_Roy_,4566
14824,drdarkpiccolo,4159
10709,SnooRegrets7769,3708
10425,ShortChecker,3090
16140,jockeposener,2968
17352,nobjos,2914


Author that wrote the most posts

In [378]:
posts.value_counts('author')

author
OPINION_IS_UNPOPULAR    50
BackgroundProgram389    42
VisualMod               41
joinfolks               35
DerekZ1985              33
                        ..
Realnickp23              1
ReapWhirlwind            1
DarkSky83                1
Reasonable-Border325     1
zztop610                 1
Length: 19791, dtype: int64

Authors that posted the most rockets

In [464]:
async def get_top_authors_for_regex(regex):
    # POSTS
    regex_posts = await beneath.query_warehouse(fr"""
    SELECT created_on, title, text, author, permalink, array_length(regexp_extract_all(title, r"{regex}")) as num_title_regex, array_length(regexp_extract_all(text, r"{regex}")) as num_body_regex
    FROM `examples/reddit/r-wallstreetbets-posts`
    WHERE timestamp_trunc(created_on, month) = timestamp("2021-06-01") AND
        array_length(regexp_extract_all(text, r"{regex}"))+array_length(regexp_extract_all(title, r"{regex}")) > 0
    """)
    regex_posts['num_regex'] = regex_posts['num_title_regex'] + regex_posts['num_body_regex']
    regex_posts_agg = regex_posts.groupby('author')['num_regex'] \
        .sum() \
        .reset_index() \
        .sort_values('num_regex', ascending=False)

    # COMMENTS
    regex_comments = await beneath.query_warehouse(fr"""
    SELECT created_on, text, author, permalink, array_length(regexp_extract_all(text, r"{regex}")) as num_regex
    FROM `examples/reddit/r-wallstreetbets-comments`
    WHERE timestamp_trunc(created_on, month) = timestamp("2021-06-01") AND
        array_length(regexp_extract_all(text, r"{regex}")) > 0
    """)
    regex_comments_agg = regex_comments.groupby('author')['num_regex'] \
        .sum() \
        .reset_index() \
        .sort_values('num_regex', ascending=False)

    # combine
    regex_df = regex_posts_agg.merge(regex_comments_agg, how="outer", on="author", suffixes=('_posts', '_comments')).fillna(0)
    regex_df['total_regex'] = regex_df['num_regex_posts'] + regex_df['num_regex_comments']
    regex_df = regex_df.sort_values('total_regex', ascending=False)
    
    return regex_df

In [465]:
regex = "\\x{1F680}" # rockets
rocket_authors = await get_top_authors_for_regex(regex)

,author,num_regex_posts,num_regex_comments,total_regex
3309,mateace,0.0,2717.0,2717.0
779,unipaulie,5.0,1746.0,1751.0
3310,LordTender,0.0,1039.0,1039.0
3311,bobdavid2223,0.0,1033.0,1033.0
3312,Hedgies_suck,0.0,1022.0,1022.0


Posted the most diamond hands

In [466]:
regex = "\\x{1F48E}" # diamonds
diamond_authors = await get_top_authors_for_regex(regex)
diamond_authors.head(10)

,author,num_regex_posts,num_regex_comments,total_regex
888,Sc0ttyMinz,0.0,710.0,710.0
889,Tazzer57,0.0,366.0,366.0
0,FriendlyhoodKomrad,330.0,0.0,330.0
890,StonedTurtle420710,0.0,323.0,323.0
891,Da-Agent,0.0,310.0,310.0
892,Objective-Dance-9438,0.0,309.0,309.0
893,chefrn99,0.0,239.0,239.0
894,JVP192,0.0,203.0,203.0
895,Josienby_,0.0,202.0,202.0
896,4181N0,0.0,197.0,197.0


In [ ]:
# - Wrote the most words
# - Wrote the most comments

## Stock mentions

In [267]:
daily_mentions = await beneath.query_warehouse("""
    SELECT symbol, timestamp_trunc(timestamp, day) as day, count(*) as num_mentions
    FROM `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions`
    WHERE timestamp_trunc(timestamp, month) = timestamp("2021-06-01")
    GROUP BY symbol, day
    ORDER BY day, num_mentions desc
""")

In [318]:
top_symbols_df = daily_mentions.loc[[symbol not in BLACKLIST for symbol in daily_mentions['symbol']]] \
    .groupby('symbol')['num_mentions'] \
    .sum() \
    .reset_index() \
    .sort_values('num_mentions', ascending=False)
top_symbols = top_symbols_df['symbol'][:100]

In [347]:
fig = px.bar(top_symbols_df[0:10], x="num_mentions", y="symbol", text="num_mentions", color='symbol', orientation='h', title='Most mentioned symbols in June', labels={'num_mentions': 'Mentions', 'symbol': 'Symbol'})
fig.update_layout(showlegend=False)
fig.show()

In [278]:
fig = px.line(daily_mentions[daily_mentions['symbol'].isin(top_symbols[:10])], 
              x="day", y="num_mentions", line_group="symbol", color="symbol",
              title="Mentions by day", labels={'num_mentions': 'mentions'})
fig.show()

In [279]:
symbol_flair = await beneath.query_warehouse("""
    SELECT symbol, timestamp_trunc(created_on, day) as day, flair, count(*) as num_posts
    FROM `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions` m
    JOIN `examples/reddit/r-wallstreetbets-posts` p ON m.post_id = p.id
    WHERE timestamp_trunc(created_on, month) = timestamp("2021-06-01")
    GROUP BY symbol, day, flair
""")

In [295]:
flair_dist = symbol_flair.groupby('flair').sum('num_posts').reset_index().sort_values('num_posts', ascending=False)
fig = px.bar(flair_dist, x="num_posts", y="flair", color='flair', orientation="h", title='Flair distribution')
fig.show()

In [340]:
flair_daily = symbol_flair.groupby(['day', 'flair']).sum('num_posts').reset_index()
fig = px.line(flair_daily, x="day", y="num_posts", color='flair', line_group='flair', title='Flair by day')
fig.show()

In [319]:
# filter for the month's top symbols
symbol_flair = symbol_flair[symbol_flair['symbol'].isin(top_symbols)]

In [306]:
wide_flair = symbol_flair.groupby(['symbol', 'flair']) \
    .sum() \
    .reset_index() \
    .pivot_table(index=['symbol'],columns='flair', values='num_posts') \
    .fillna(0)
wide_flair['total_posts'] = wide_flair.sum(axis=1)
wide_flair['DD_percent'] = wide_flair['DD']/wide_flair['total_posts']
wide_flair['Meme_percent'] = wide_flair['Meme']/wide_flair['total_posts']
wide_flair['Gain_percent'] = wide_flair['Gain']/wide_flair['total_posts']
wide_flair['Loss_percent'] = wide_flair['Loss']/wide_flair['total_posts']
wide_flair

flair,DD,Discussion,Gain,Loss,Meme,News,Shitpost,Technical Analysis,YOLO,total_posts,DD_percent,Meme_percent,Gain_percent,Loss_percent
symbol,,,,,,,,,,,,,,
AAPL,21.0,17.0,1.0,0.0,3.0,0.0,0.0,3.0,5.0,50.0,0.420000,0.060000,0.020000,0.000000
AHT,4.0,11.0,18.0,0.0,3.0,3.0,0.0,2.0,12.0,53.0,0.075472,0.056604,0.339623,0.000000
AMC,521.0,2302.0,472.0,112.0,472.0,222.0,3.0,234.0,875.0,5213.0,0.099942,0.090543,0.090543,0.021485
AMD,27.0,44.0,5.0,3.0,0.0,10.0,0.0,2.0,18.0,109.0,0.247706,0.000000,0.045872,0.027523
AMZN,34.0,31.0,10.0,0.0,2.0,2.0,0.0,0.0,4.0,83.0,0.409639,0.024096,0.120482,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOOF,26.0,40.0,14.0,1.0,13.0,1.0,0.0,4.0,36.0,135.0,0.192593,0.096296,0.103704,0.007407
WPG,5.0,9.0,5.0,0.0,5.0,1.0,0.0,3.0,11.0,39.0,0.128205,0.128205,0.128205,0.000000
WWE,26.0,18.0,3.0,0.0,2.0,3.0,0.0,4.0,7.0,63.0,0.412698,0.031746,0.047619,0.000000


In [307]:
dd_symbols = wide_flair[wide_flair.index.isin(top_symbols)] \
    .sort_values(by=['DD_percent'], ascending=False) \
    .reset_index()
dd_symbols.head(10)

flair,symbol,DD,Discussion,Gain,Loss,Meme,News,Shitpost,Technical Analysis,YOLO,total_posts,DD_percent,Meme_percent,Gain_percent,Loss_percent
0,UPS,40.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,60.0,0.666667,0.000000,0.000000,0.000000
1,BP,34.0,18.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,58.0,0.586207,0.034483,0.034483,0.000000
2,MA,27.0,11.0,2.0,0.0,1.0,0.0,0.0,5.0,1.0,47.0,0.574468,0.021277,0.042553,0.000000
3,OI,34.0,24.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,63.0,0.539683,0.000000,0.000000,0.000000
4,GM,55.0,26.0,5.0,0.0,3.0,8.0,0.0,2.0,5.0,104.0,0.528846,0.028846,0.048077,0.000000
5,TWNK,34.0,8.0,3.0,1.0,14.0,0.0,0.0,1.0,4.0,65.0,0.523077,0.215385,0.046154,0.015385
6,IP,22.0,15.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,43.0,0.511628,0.000000,0.046512,0.023256
7,JP,25.0,22.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,53.0,0.471698,0.000000,0.018868,0.000000
8,LNG,21.0,13.0,4.0,0.0,0.0,3.0,0.0,5.0,3.0,49.0,0.428571,0.000000,0.081633,0.000000
9,SP,21.0,18.0,1.0,0.0,0.0,0.0,0.0,3.0,7.0,50.0,0.420000,0.000000,0.020000,0.000000


In [309]:
fig = px.bar(dd_symbols[0:20], x="DD_percent", y="symbol", color='symbol', orientation='h', text='DD', title='Top DD Stocks')
fig.show()

In [320]:
meme_symbols = wide_flair[wide_flair.index.isin(top_symbols)] \
    .sort_values(by=['Meme_percent'], ascending=False) \
    .reset_index()
meme_symbols.head(10)

flair,symbol,DD,Discussion,Gain,Loss,Meme,News,Shitpost,Technical Analysis,YOLO,total_posts,DD_percent,Meme_percent,Gain_percent,Loss_percent
0,BLNK,7.0,5.0,1.0,1.0,9.0,2.0,0.0,1.0,10.0,36.0,0.194444,0.250000,0.027778,0.027778
1,TWNK,34.0,8.0,3.0,1.0,14.0,0.0,0.0,1.0,4.0,65.0,0.523077,0.215385,0.046154,0.015385
2,BABA,18.0,7.0,2.0,0.0,7.0,1.0,0.0,5.0,13.0,53.0,0.339623,0.132075,0.037736,0.000000
3,WPG,5.0,9.0,5.0,0.0,5.0,1.0,0.0,3.0,11.0,39.0,0.128205,0.128205,0.128205,0.000000
4,VIAC,11.0,18.0,9.0,0.0,9.0,2.0,0.0,9.0,15.0,73.0,0.150685,0.123288,0.123288,0.000000
5,NOK,17.0,133.0,14.0,8.0,32.0,11.0,0.0,11.0,62.0,288.0,0.059028,0.111111,0.048611,0.027778
6,NKLA,13.0,23.0,4.0,0.0,6.0,4.0,0.0,3.0,5.0,58.0,0.224138,0.103448,0.068966,0.000000
7,SNDL,33.0,185.0,40.0,5.0,47.0,11.0,0.0,20.0,123.0,464.0,0.071121,0.101293,0.086207,0.010776
8,SDC,5.0,18.0,6.0,3.0,6.0,2.0,0.0,2.0,19.0,61.0,0.081967,0.098361,0.098361,0.049180
9,WOOF,26.0,40.0,14.0,1.0,13.0,1.0,0.0,4.0,36.0,135.0,0.192593,0.096296,0.103704,0.007407


In [321]:
fig = px.bar(meme_symbols[0:10], x="Meme_percent", y="symbol", color='symbol', orientation='h', title='Top Meme Stocks')
fig.show()

In [332]:
# get BLNK memes
blnk_memes = await beneath.query_warehouse("""
    SELECT symbol, timestamp_trunc(created_on, day) as day, flair, id, permalink
    FROM `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions` m
    JOIN `examples/reddit/r-wallstreetbets-posts` p ON m.post_id = p.id
    WHERE timestamp_trunc(created_on, month) = timestamp("2021-06-01") AND
    symbol="BLNK" AND 
    flair="Meme"
""")

In [335]:
blnk_memes
# most (all?) of these were deleted by mods

,symbol,day,flair,id,permalink,@meta.timestamp
0,BLNK,2021-06-03 00:00:00+00:00,Meme,nrcewp,/r/wallstreetbets/comments/nrcewp/shorting_bln...,2021-06-29 17:33:09.526000+00:00
1,BLNK,2021-06-03 00:00:00+00:00,Meme,nremfo,/r/wallstreetbets/comments/nremfo/time_to_shor...,2021-06-29 17:33:09.526000+00:00
2,BLNK,2021-06-03 00:00:00+00:00,Meme,nrftwz,/r/wallstreetbets/comments/nrftwz/anyone_ready...,2021-06-29 17:33:09.526000+00:00
3,BLNK,2021-06-06 00:00:00+00:00,Meme,ntnxie,/r/wallstreetbets/comments/ntnxie/amcx_still_o...,2021-06-29 17:33:09.526000+00:00
4,BLNK,2021-06-07 00:00:00+00:00,Meme,nufcgg,/r/wallstreetbets/comments/nufcgg/stop_buying_...,2021-06-29 17:33:09.526000+00:00
5,BLNK,2021-06-08 00:00:00+00:00,Meme,nvbmhi,/r/wallstreetbets/comments/nvbmhi/blnk_is_at_3...,2021-06-29 17:33:09.526000+00:00
6,BLNK,2021-06-08 00:00:00+00:00,Meme,nvbp01,/r/wallstreetbets/comments/nvbp01/blnk_is_at_3...,2021-06-29 17:33:09.526000+00:00
7,BLNK,2021-06-02 00:00:00+00:00,Meme,nqtj2o,/r/wallstreetbets/comments/nqtj2o/the_meme_etf...,2021-06-29 17:33:09.526000+00:00
8,BLNK,2021-06-02 00:00:00+00:00,Meme,nqwrwz,/r/wallstreetbets/comments/nqwrwz/puts_on_blnk...,2021-06-29 17:33:09.526000+00:00


In [322]:
gain_symbols = wide_flair[wide_flair.index.isin(top_symbols)] \
    .sort_values(by=['Gain_percent'], ascending=False) \
    .reset_index()
gain_symbols.head(10)

flair,symbol,DD,Discussion,Gain,Loss,Meme,News,Shitpost,Technical Analysis,YOLO,total_posts,DD_percent,Meme_percent,Gain_percent,Loss_percent
0,AHT,4.0,11.0,18.0,0.0,3.0,3.0,0.0,2.0,12.0,53.0,0.075472,0.056604,0.339623,0.000000
1,GTT,3.0,13.0,9.0,0.0,1.0,3.0,0.0,1.0,9.0,39.0,0.076923,0.025641,0.230769,0.000000
2,TRCH,9.0,31.0,19.0,3.0,1.0,6.0,0.0,5.0,16.0,90.0,0.100000,0.011111,0.211111,0.033333
3,RMED,10.0,14.0,10.0,0.0,3.0,2.0,0.0,2.0,7.0,48.0,0.208333,0.062500,0.208333,0.000000
4,SPCE,28.0,103.0,50.0,4.0,14.0,11.0,0.0,11.0,38.0,259.0,0.108108,0.054054,0.193050,0.015444
5,GEO,8.0,26.0,16.0,0.0,5.0,5.0,0.0,6.0,21.0,87.0,0.091954,0.057471,0.183908,0.000000
6,SENS,37.0,44.0,27.0,1.0,5.0,7.0,0.0,9.0,32.0,162.0,0.228395,0.030864,0.166667,0.006173
7,IVR,5.0,26.0,11.0,1.0,1.0,1.0,0.0,1.0,21.0,67.0,0.074627,0.014925,0.164179,0.014925
8,NVDA,18.0,18.0,9.0,1.0,0.0,2.0,0.0,2.0,5.0,55.0,0.327273,0.000000,0.163636,0.018182
9,ASTS,6.0,9.0,8.0,2.0,2.0,1.0,0.0,8.0,14.0,50.0,0.120000,0.040000,0.160000,0.040000


In [329]:
fig = px.bar(gain_symbols[0:5], x="Gain_percent", y="symbol", color='symbol', orientation='h', title='Top Gain Stocks')
fig.show()

In [326]:
loss_symbols = wide_flair[wide_flair.index.isin(top_symbols)] \
    .sort_values(by=['Loss_percent'], ascending=False) \
    .reset_index()
loss_symbols.head(10)

flair,symbol,DD,Discussion,Gain,Loss,Meme,News,Shitpost,Technical Analysis,YOLO,total_posts,DD_percent,Meme_percent,Gain_percent,Loss_percent
0,ORPH,0.0,27.0,6.0,4.0,1.0,0.0,0.0,1.0,10.0,49.0,0.000000,0.020408,0.122449,0.081633
1,QS,6.0,15.0,2.0,2.0,1.0,2.0,0.0,3.0,5.0,36.0,0.166667,0.027778,0.055556,0.055556
2,TD,20.0,57.0,2.0,5.0,1.0,7.0,0.0,4.0,4.0,100.0,0.200000,0.010000,0.020000,0.050000
3,SDC,5.0,18.0,6.0,3.0,6.0,2.0,0.0,2.0,19.0,61.0,0.081967,0.098361,0.098361,0.049180
4,GSAT,4.0,14.0,3.0,2.0,4.0,4.0,0.0,2.0,10.0,43.0,0.093023,0.093023,0.069767,0.046512
5,PSFE,16.0,9.0,2.0,3.0,1.0,0.0,0.0,2.0,32.0,65.0,0.246154,0.015385,0.030769,0.046154
6,BB,246.0,1085.0,230.0,130.0,252.0,66.0,0.0,114.0,845.0,2968.0,0.082884,0.084906,0.077493,0.043801
7,NAKD,3.0,41.0,15.0,4.0,8.0,2.0,0.0,5.0,20.0,98.0,0.030612,0.081633,0.153061,0.040816
8,ASTS,6.0,9.0,8.0,2.0,2.0,1.0,0.0,8.0,14.0,50.0,0.120000,0.040000,0.160000,0.040000
9,UWMC,86.0,135.0,35.0,19.0,20.0,9.0,0.0,45.0,142.0,491.0,0.175153,0.040733,0.071283,0.038697


In [328]:
fig = px.bar(loss_symbols[0:5], x="Loss_percent", y="symbol", color='symbol', orientation='h', title='Top Loss Stocks')
fig.show()